In [1]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import pandas as pd
%matplotlib inline
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import ast

data = pd.read_csv('data.csv')
data = data.drop(['street', 'zipcode', 'mls-id', 'private pool', 'MlsId', 'propertyType', 'status', 'city'], axis=1)
data.drop([330468, 377183], inplace = True)

def data_func1(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][0].get('factValue')

def data_func2(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][1].get('factValue')

def data_func3(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][2].get('factValue')

def data_func4(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][3].get('factValue')

def data_func5(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][4].get('factValue')

def data_func6(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][5].get('factValue')

def data_func7(x):
    a = ast.literal_eval(x)
    return a['atAGlanceFacts'][6].get('factValue')

def data_func8(x):
    m = []
    a = ast.literal_eval(x)
    a = dict(a[0])
    k = a['data'].get('Distance')
    for i in k:
        h = re.findall(r'\d+',i)
        if len(h) == 0:
            continue
        if len(h) == 1:
            m.append(float(h[0]))
            continue
        d = h[0] + '.' + h[1]
        m.append(float(d))
    if len(m) == 0:
        return 0
    else:
        return min(m)
    
def state(x):
    if x in top:
        return 2
    if x in mid:
        return 1
    return 0
    
top = ['WA', 'NH', 'MN', 'UT', 'VT', 'MD', 'VA', 'MA', 'NE', 'CO', 'WI', 'NJ', 'FL', 'IA', 'ND', 'ID']
mid = ['GA', 'NC', 'CA', 'SD', 'CT', 'KS', 'DE', 'HI', 'NY', 'RI', 'OR', 'MO', 'MT', 'TN', 'WY', 'ME']
data['states'] = data['state'].apply(state)
data['distance'] = data['schools'].apply(data_func8)
data['built'] = data['homeFacts'].apply(data_func1)
data['heating'] = data['homeFacts'].apply(data_func3)
data['cooling'] = data['homeFacts'].apply(data_func4)
data['parking'] = data['homeFacts'].apply(data_func5)
data['lotsize'] = data['homeFacts'].apply(data_func6)
data['price'] = data['homeFacts'].apply(data_func7)
data = data.drop(['homeFacts', 'schools', 'state'], axis = 1)

def func_pool(x):
    if type(x) != int:
        x = x.replace(x, '1')
    return int(x)

def func_built(x):
    if x == 'No Data':
        x = 1980
    return x

data['built'] = data['built'].fillna(1980).apply(func_built)
data['PrivatePool'] = data['PrivatePool'].fillna(0).apply(func_pool)
data['fireplace'] = data['fireplace'].fillna(0).apply(func_pool)
data['heating'] = data['heating'].fillna(0).apply(func_pool)
data['cooling'] = data['cooling'].fillna(0).apply(func_pool)
data['parking'] = data['parking'].fillna(0).apply(func_pool)
pool_dummies = pd.get_dummies(data["PrivatePool"], prefix="Pool", dummy_na=False)
data = pd.concat([data, pool_dummies], axis=1)
fire_dummies = pd.get_dummies(data["fireplace"], prefix="fire", dummy_na=False)
data = pd.concat([data, fire_dummies], axis=1)
heat_dummies = pd.get_dummies(data["heating"], prefix="heat", dummy_na=False)
data = pd.concat([data, heat_dummies], axis=1)
cool_dummies = pd.get_dummies(data["cooling"], prefix="cool", dummy_na=False)
data = pd.concat([data, cool_dummies], axis=1)
park_dummies = pd.get_dummies(data["parking"], prefix="park", dummy_na=False)
data = pd.concat([data, park_dummies], axis=1)
data = data.drop(['fireplace', 'PrivatePool', 'heating', 'cooling', 'parking'], axis = 1)

def func_cost(x):
    if type(x) != int:
        x = re.findall(r'\d+',x)
        if len(x) > 2:
            x = x[0] + x[1] + x[2]
        elif len(x) > 1:
            x = x[0] + x[1]
        elif len(x) == 1:
            x = x[0]
        if not x:
            x = 7500
    return int(x)

def func_stor(x):
    if type(x) != int:
        x = re.findall(r'\d+',x)
        if (len(x) > 1):
            x = x[0]
        elif len(x) == 1:
            x = x[0]
        if not x:
            x = 1
    return int(x)

def func_area(x):
    if type(x) != int:
        x = re.findall(r'\d+',x)
        if len(x) > 1:
            x = x[0] + x[1]
        elif len(x) == 1:
            x = x[0]
        if not x:
            x = 100
    return int(x)

data['target'] = data['target'].fillna(7500).apply(func_cost)
data['stories'] = data['stories'].fillna(0).apply(func_stor)
data['beds'] = data['beds'].fillna(0).apply(func_stor)
data['baths'] = data['baths'].fillna(1).apply(func_stor)
data['sqft'] = data['sqft'].fillna(1900).apply(func_area)
data['lotsize'] = data['lotsize'].fillna(1900).apply(func_area)
data['price'] = data['price'].fillna(100).apply(func_area)
data['target'] = pd.to_numeric(data['target'])
data['stories'] = pd.to_numeric(data['stories'])
data['lotsize'] = pd.to_numeric(data['lotsize'])
data['sqft'] = pd.to_numeric(data['sqft'])
data['beds'] = pd.to_numeric(data['beds'])
data['baths'] = pd.to_numeric(data['baths'])
data['price'] = pd.to_numeric(data['price'])
data['built'] = pd.to_numeric(data['built'])
data = data[data['baths'] < 60]
data = data[data['beds'] < 60]
data = data[data['stories'] < 100]
data['built'] = data['built'].fillna(1980).apply(func_built)
X = data.drop('target', axis = 1)
y = data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=23, test_size=0.7)
regr_rf = RandomForestRegressor(n_estimators = 120, 
                             random_state = 23, 
                             max_depth = 11, 
                             n_jobs=-1, 
                             min_samples_split = 5, 
                             min_samples_leaf = 3)
regr_rf.fit(X_train, y_train)
rf_pred = regr_rf.predict(X_test)
regr_gb = GradientBoostingRegressor(random_state = 23, n_estimators = 260, max_depth = 4)
regr_gb.fit(X_train, y_train)
gb_pred = regr_gb.predict(X_test)
y_pred = 0.6*rf_pred + 0.4*gb_pred

#submission = pd.DataFrame({"target": y_pred})
#submission.to_csv("submission.csv", index=False)